In [7]:
import requests
from bs4 import BeautifulSoup as BS

In [8]:
def get_m_content(year, start):
    url = "https://www.imdb.com/search/title?title_type=feature&year={}-01-01,{}-12-31&start={}&ref_=adv_nxt".format(year, year, start)
    print(url)
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    page = requests.get(url, headers = headers, timeout=5)
    page.status_code
    soup = BS(page.content, 'html.parser')
    m_content = soup.findAll('div', class_ = 'lister-item-content')
    return m_content

In [9]:
import re

def parse_movie(m_content):
    m = {}

    m_header = m_content.find('h3', class_ = 'lister-item-header')
    if m_header.find('a'):
        m['title'] = m_header.find('a').get_text()
    if m_header.find('span', class_ = 'lister-item-year'):
        m['year'] = m_header.find('span', class_ = 'lister-item-year').get_text()
        if m['year']:
            m['year'] = re.findall('\d+', m['year'])

    m_muted = m_content.find('p', class_ = 'text-muted')
    if m_muted.find('span', class_ = 'certificate'):
        m['certificate'] = m_muted.find('span', class_ = 'certificate').get_text()
    if m_muted.find('span', class_ = 'runtime'):
        m['runtime'] = m_muted.find('span', class_ = 'runtime').get_text()
    if m_muted.find('span', class_ = 'genre'):
        m['genre'] = m_muted.find('span', class_ = 'genre').get_text().strip()
        m['genre'] = m['genre'].split(', ')

    m_ratings = m_content.find('div', class_ = 'ratings-bar')
    if m_ratings:
        if m_ratings.find('div', class_ = 'ratings-imdb-rating'):
            m['imdb_rating'] = float(m_ratings.find('div', class_ = 'ratings-imdb-rating').get_text().strip())
        if m_ratings.find('div', class_ = 'ratings-metascore'):
            m['metascore'] = m_ratings.find('div', class_ = 'ratings-metascore').get_text().strip()
            m['metascore'] = int(m['metascore'].split('\n')[0].strip())

    m_visible = m_content.find('p', class_ = 'sort-num_votes-visible')
    if m_visible:
        m_visible_children = m_visible.findChildren()
        if len(m_visible_children)>=2:
            m['votes'] = int(m_visible_children[1]['data-value'].replace(',',''))
            if len(m_visible_children)>=5:
                m['gross'] = int(m_visible_children[4]['data-value'].replace(',',''))

    return m

In [ ]:
import time

all_2018_movies = []
for i in list(range(1,10000,50)):
    m_content = get_m_content(2018, i)
    time.sleep(5)
    
    for movie in m_content:
        all_movies.append(parse_movie(movie))
        
all_2018_movies

In [ ]:
import time

all_2019_movies = []
for i in list(range(1,6600,50)):
    m_content = get_m_content(2019, i)
    time.sleep(10)
    
    for movie in m_content:
        all_movies.append(parse_movie(movie))
        
all_2019_movies

https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=1&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=51&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=101&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=151&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=201&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=251&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=301&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=351&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=401&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12

In [ ]:
print(len(all_2018_movies))
print(len(all_2019_movies))

In [6]:
import pandas as pd

df_2018 = pd.DataFrame(all_2018_movies)
df_2018.to_csv('imdb_2018.csv')
df_2018.head()

,certificate,genre,gross,imdb_rating,metascore,runtime,title,votes,year
0,NaN,"[Comedy, Drama]",NaN,NaN,NaN,NaN,Once Upon a Time in Hollywood,NaN,[2019]
1,R,"[Horror, Thriller]",77075050.0,7.5,81.0,116 min,Us,33380.0,[2019]
2,PG-13,"[Action, Adventure, Sci-Fi]",323646033.0,7.2,64.0,123 min,Captain Marvel,190562.0,[2019]
3,R,"[Action, Adventure, Crime]",NaN,6.5,61.0,125 min,Triple Frontier,47069.0,[2019]
4,NaN,"[Action, Adventure, Fantasy]",NaN,NaN,NaN,182 min,Avengers: Endgame,NaN,[2019]


In [7]:
df_2019 = pd.DataFrame(all_movies)
df_2019.to_csv('imdb_2019.csv')
df_2019.head()